In [1]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# Reading Data
#df_online_retail = pd.read_csv('../../00_Data/data.csv', encoding= 'unicode_escape', na_values='nan')
#df_online_retail_xlsx = pd.read_excel('../00_Data/online_retail.xlsx', sheet_name=0)
df_online_retail = pd.read_excel('../../00_Data/online_retail_II.xlsx', sheet_name=0)
#df_rec_sys_order = pd.read_excel('../00_Data/Rec_sys_data.xlsx', sheet_name=0)
#df_rec_sys_customer = pd.read_excel('../00_Data/Rec_sys_data.xlsx', sheet_name=1)
#df_rec_sys_product = pd.read_excel('../00_Data/Rec_sys_data.xlsx', sheet_name=2)

# 1. Online Retail Dataset

## 1.1 Initial Data Check

In [4]:
# Find out number of missing values per column
df_online_retail.isna().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [6]:
# We need to drop missing values in CustomerID
df_online_retail = df_online_retail.dropna(subset=['Customer ID'])

# Sanity check 
df_online_retail.isna().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [7]:
# First check Data Types 
df_online_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      417534 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 28.7+ MB


In [13]:
# Since we are going to encode the labels for User and Items, we need to make sure that we convert those to strings 
df_online_retail['Customer ID'] = df_online_retail['Customer ID'].astype(str)
df_online_retail['StockCode'] = df_online_retail['StockCode'].astype(str)

# We can also convert Invoice Date into Date time
df_online_retail['InvoiceDate'] = pd.to_datetime(df_online_retail['InvoiceDate'])
df_online_retail.info()

/var/folders/bs/ht_t1wp94tl3wt5xhchcxq2m0000gn/T/ipykernel_79624/842562789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_online_retail['Customer ID'] = df_online_retail['Customer ID'].astype(str)
/var/folders/bs/ht_t1wp94tl3wt5xhchcxq2m0000gn/T/ipykernel_79624/842562789.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_online_retail['StockCode'] = df_online_retail['StockCode'].astype(str)
/var/folders/bs/ht_t1wp94tl3wt5xhchcxq2m0000gn/T/ipykernel_79624/842562789.py:6: SettingWithCopyWarning: 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 417534 entries, 0 to 525460
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  object        
 7   Country      417534 non-null  object        
 8   purchased    417534 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 31.9+ MB


In [9]:
# Last but not least, we will need a column, which indicates that an item was bought by a user. We are assuming, this column is binary - 1 means bought, 0 means no interaction
df_online_retail['purchased'] = 1
df_online_retail.head()

/var/folders/bs/ht_t1wp94tl3wt5xhchcxq2m0000gn/T/ipykernel_79624/243351604.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_online_retail['purchased'] = 1


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,purchased
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,1
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,1
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,1
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,1
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,1


In [14]:
df_online_retail.groupby('Customer ID').size().sort_values()

Customer ID
15233.0       1
13128.0       1
15913.0       1
18174.0       1
15997.0       1
           ... 
12748.0    2665
14156.0    2710
14606.0    3927
17841.0    5114
14911.0    5710
Length: 4383, dtype: int64

In [15]:
# User & Item  Label Encoder 

# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Set up Objects for User and Item Column
le1 = LabelEncoder()
le2 = LabelEncoder()

# Fit the encoders to the columns in question
le1.fit(df_online_retail['StockCode'])
le2.fit(df_online_retail['Customer ID'])

# Transform dataframe with encoded labels
df_online_retail['StockCode'] = le1.transform(df_online_retail['StockCode'])
df_online_retail['Customer ID'] = le2.transform(df_online_retail['Customer ID'])

# Check Change 
df_online_retail.head()

/var/folders/bs/ht_t1wp94tl3wt5xhchcxq2m0000gn/T/ipykernel_79624/2673937164.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_online_retail['StockCode'] = le1.transform(df_online_retail['StockCode'])
/var/folders/bs/ht_t1wp94tl3wt5xhchcxq2m0000gn/T/ipykernel_79624/2673937164.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_online_retail['Customer ID'] = le2.transform(df_online_retail['Customer ID'])


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,purchased
0,489434,3423,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,507,United Kingdom,1
1,489434,2740,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,507,United Kingdom,1
2,489434,2742,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,507,United Kingdom,1
3,489434,1258,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,507,United Kingdom,1
4,489434,617,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,507,United Kingdom,1


## 1.2. Filter out Users

In [17]:
# Count the number of rows per user
rows_per_user = df_online_retail.groupby('Customer ID').size()

# Create a new DataFrame with the desired columns and reset the index
df_rows_per_user = rows_per_user.reset_index(name='Purchases')

# Rename the column to match the original DataFrame
df_rows_per_user = df_rows_per_user.rename(columns={'Customer ID': 'Customer ID'})

# Display the new DataFrame
df_rows_per_user.sort_values(by='Purchases', ascending=False)

,Customer ID,Purchases
1869,1869,5710
4058,4058,5114
1631,1631,3927
1291,1291,2710
254,254,2665
...,...,...
181,181,1
1266,1266,1
1282,1282,1
3006,3006,1


In [18]:
# Find out a Threshold to cut
print("Users with less than 13 Purchases \t:", len(df_rows_per_user[df_rows_per_user['Purchases']<13]))
print("Users with less than 15 Purchases \t:", len(df_rows_per_user[df_rows_per_user['Purchases']<15]))
print("Users with less than 20 Purchases \t:", len(df_rows_per_user[df_rows_per_user['Purchases']<20]))
print("Users with less than 50 Purchases \t:", len(df_rows_per_user[df_rows_per_user['Purchases']<50]))

Users with less than 13 Purchases 	: 759
Users with less than 15 Purchases 	: 895
Users with less than 20 Purchases 	: 1192
Users with less than 50 Purchases 	: 2353


In [20]:
# Create a list of users with less than 20 Purchases
customer_less_20 = df_rows_per_user[df_rows_per_user['Purchases']<20]['Customer ID'].tolist()

# Filter out these users from the initial DataFrame df_online_retail
df_online_retail_20 = df_online_retail[~df_online_retail['Customer ID'].isin(customer_less_20)]

# Show shape of new DataFrame
df_online_retail_20.shape

(406087, 9)

In [21]:
# How many rows less?
df_online_retail.shape[0] - df_online_retail_20.shape[0]

11447

## 1.3. Data Split

In [22]:
from recommenders.datasets.sparse import AffinityMatrix

In [26]:
#to use standard names across the analysis 
header = {
        "col_user": "Customer ID",
        "col_item": "StockCode",
        "col_rating": "purchased",
    }

#instantiate the sparse matrix generation  
am = AffinityMatrix(df = df_online_retail_20, col_user='Customer ID', col_item='StockCode', col_rating='purchased')

#obtain the sparse matrix 
X, _, _ = am.gen_affinity_matrix()

# check that the generated matrix has the correct dimensions
assert (X.shape[0] == df_online_retail_20['Customer ID'].unique().shape[0]) & (
        X.shape[1] == df_online_retail_20.StockCode.unique().shape[0]
    )

In [27]:
from recommenders.datasets.python_splitters import numpy_stratified_split
Xtr, Xtst = numpy_stratified_split(X, ratio=0.8, seed=1)

In [28]:
print('train matrix size', Xtr.shape)
print('test matrix size', Xtst.shape)

train matrix size (3191, 4024)
test matrix size (3191, 4024)


In [29]:
count_nonzero_train = np.sum(Xtr != 0)
count_nonzero_test = np.sum(Xtst!=0)
print('Elements in train: \t', count_nonzero_train)
print('Elements in test:\t', count_nonzero_test)

Elements in train: 	 214704
Elements in test:	 50398


In [30]:
# Turn all values to 5 that are above 0 
Xtr = np.where(Xtr > 0, 5, Xtr)
#Xtr = np.where(Xtr ==0, 1, Xtr)

In [31]:
# Turn all values to 5 that are above 0 
Xtst = np.where(Xtst > 0, 5, Xtst)
#Xtst = np.where(Xtst == 0, 1, Xtst)

## 1.3. Model

In [32]:
import logging
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

#RBM 
from recommenders.models.rbm.rbm import RBM
from recommenders.utils.timer import Timer
from recommenders.utils.plot import line_graph

# Evaluation
from recommenders.evaluation.python_evaluation import (
    precision_at_k,
    recall_at_k,
    map_at_k,
    auc
)


In [442]:
try:
    del(model)
except:
    pass

In [33]:
#First we initialize the model class
model = RBM(
    possible_ratings=np.setdiff1d(np.unique(Xtr), np.array([0])),
    visible_units=Xtr.shape[1],
    hidden_units=100,
    training_epoch=30,
    minibatch_size=10,
    keep_prob=0.8,
    with_metrics=True,
    seed=1,
    learning_rate=0.0001
)

2023-03-16 12:44:32.561589: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 12:44:32.603284: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


In [34]:
#Model Fit
with Timer() as train_time:
    model.fit(Xtr)

print("Took {:.2f} seconds for training.".format(train_time.interval))

Took 36.55 seconds for training.


In [35]:
# number of top score elements to be recommended  
K = 20

# Model prediction on the test set Xtst.
with Timer() as prediction_time:
    top_k =  model.recommend_k_items(Xtst, top_k=K)

print("Took {:.2f} seconds for prediction.".format(prediction_time.interval))

Took 0.81 seconds for prediction.


In [36]:
# Map Back Xtst & Top_k 
top_k_df = am.map_back_sparse(top_k, kind = 'prediction')
test_df = am.map_back_sparse(Xtst, kind = 'ratings')

In [37]:
# Drop Customer ID because non existent
top_k_df = top_k_df.drop(top_k_df[top_k_df['Customer ID'] == 0].index)

In [38]:
#collection of evaluation metrics for later use
def ranking_metrics(
    data_size,
    data_true,
    data_pred,
    K
):
    eval_precision = precision_at_k(data_true, data_pred, col_user="Customer ID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k= K)

    eval_recall = recall_at_k(data_true, data_pred, col_user="Customer ID", col_item="StockCode", 
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k= K)
    eval_map = map_at_k(data_true, data_pred, col_user="Customer ID", col_item="StockCode", 
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k= K)

    df_result = pd.DataFrame(
        {   "Dataset": data_size,
            "K": K,
            "Precision@k": eval_precision,
            "Recall@k": eval_recall,
            "MAP": eval_map,
        }, 
        index=[0]
    )

    return df_result

In [39]:
eval_at_10= ranking_metrics(
    data_size="Online Retail",
    data_true=test_df,
    data_pred=top_k_df,
    K=20
)

eval_at_10

,Dataset,K,Precision@k,Recall@k,MAP
0,Online Retail,20,0.012476,0.016655,0.003473


In [40]:
top_k_df[top_k_df['Customer ID']==300].sort_values('StockCode')

,Customer ID,StockCode,prediction
4541,300,22,5.0
4546,300,23,5.0
4542,300,24,5.0
4550,300,210,5.0
4558,300,213,5.0
4556,300,862,5.0
4540,300,863,5.0
4554,300,1838,5.0
4553,300,1865,5.0
4547,300,1866,5.0


In [41]:
test_df[test_df['Customer ID']==300].sort_values('StockCode')

,Customer ID,StockCode,purchased
3407,300,517,5
3406,300,1156,5
3408,300,1201,5
3405,300,1577,5
3404,300,1738,5
